In [ ]:
! git clone https://github.com/adc257/info4940-sitcom.git
#from google.colab import drive
#drive.mount('/content/drive')


Cloning into 'info4940-sitcom'...
remote: Enumerating objects: 1122, done.
remote: Counting objects: 100% (1122/1122), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 1122 (delta 972), reused 1046 (delta 917), pack-reused 0
Receiving objects: 100% (1122/1122), 14.75 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (972/972), done.


In [ ]:
#CODE FOR LATER USE #CODE that extracts all humor dialogues in the whole SEASON 1
import os
import json

root_dir = "/content/info4940-sitcom/cleaned-data/S1"
episode_humor_data = {}

# Iterate over each episode file in the S1 folder
for episode_file in os.listdir(root_dir):
    episode_path = os.path.join(root_dir, episode_file)
    if os.path.isfile(episode_path):
        with open(episode_path, 'r') as file:
            try:
                data = json.load(file)
                humor_dialogues = [dialogue for timestamp, dialogue in data.items() if dialogue.get('isHumor', False)]
                episode_humor_data[episode_file] = humor_dialogues
            except json.JSONDecodeError:
                print(f"Error decoding JSON from file: {episode_path}")
            except Exception as e:
                print(f"Unexpected error processing file {episode_path}: {e}")

# Process the humor data for each episode as needed
for episode, humor_dialogues in episode_humor_data.items():
    print(f"Episode: {episode}, Number of Humor Dialogues: {len(humor_dialogues)}")

#for dialogue in humor_dialogues[:4]:
#        print(dialogue)
#        print("\n")

Episode: The Big Bang_S0117.json, Number of Humor Dialogues: 102
Episode: The Big Bang_S0109.json, Number of Humor Dialogues: 95
Episode: The Big Bang_S0115.json, Number of Humor Dialogues: 130
Episode: The Big Bang_S0114.json, Number of Humor Dialogues: 97
Episode: The Big Bang_S0103.json, Number of Humor Dialogues: 109
Episode: The Big Bang_S0111.json, Number of Humor Dialogues: 123
Episode: The Big Bang_S0101.json, Number of Humor Dialogues: 125
Episode: The Big Bang_S0106.json, Number of Humor Dialogues: 91
Episode: The Big Bang_S0104.json, Number of Humor Dialogues: 114
Episode: The Big Bang_S0107.json, Number of Humor Dialogues: 126
Episode: The Big Bang_S0113.json, Number of Humor Dialogues: 111
Episode: The Big Bang_S0112.json, Number of Humor Dialogues: 119
Episode: The Big Bang_S0108.json, Number of Humor Dialogues: 116
Episode: The Big Bang_S0110.json, Number of Humor Dialogues: 100
Episode: The Big Bang_S0116.json, Number of Humor Dialogues: 94
Episode: The Big Bang_S0102.j

In [ ]:
#TASK: Humor detection using rag on S1E5

import os
import json

def create_episode_dataset(file_path):
    with open(file_path, 'r') as file:
        episode_data = json.load(file)

    dataset = []

    for timestamp, dialogue in episode_data.items():
        # Add current dialogue and its humor classification to the dataset
        dataset.append({
            "episode": file_path.split('/')[-1],
            "timestamp": timestamp,
            "scene": dialogue['Scene'],
            "recipients": dialogue['Recipients'],
            "speaker": dialogue['Speaker'],
            "dialogue": dialogue['Dialogue'],
            "is_joke": dialogue.get('isHumor', False)
        })

    return dataset

# Path to your episode 5 file
episode_file = "/content/info4940-sitcom/cleaned-data/S1/The Big Bang_S0105.json"
episode_dataset = create_episode_dataset(episode_file)

# Example: Print the first few entries from the dataset
#for entry in episode_dataset[:4]:
#    print(entry)

#create a textual piece that combines all the scenes and dialogues

formatted_entries = []
for entry in episode_dataset:
    formatted_entry = (
        f"The time is {entry['timestamp']}. "
        f"The scene is {entry['scene']}. "
        f"The recipients are {', '.join(entry['recipients'])}. "
        f"The speaker is {entry['speaker']}. "
        f"{entry['speaker']} says \"{entry['dialogue']}\"\n"
    )
    formatted_entries.append(formatted_entry)

#save as text file
output_file_path = "/content/info4940-sitcom/episode_5_formatted.txt"
with open(output_file_path, 'w') as file:
    for formatted_entry in formatted_entries:
        file.write(formatted_entry)

print(f"Formatted entries saved to {output_file_path}")

#for formatted_entry in formatted_entries[:4]:
#    print(formatted_entry)

Formatted entries saved to /content/info4940-sitcom/episode_5_formatted.txt


In [ ]:
#RAG IMPLEMENTATION ON PREVIOUS CODE CHUNK - formatted_entry

#Installing libraries

!pip install torch transformers
from transformers import T5Tokenizer, T5EncoderModel
import torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

### **LANGCHAIN ATTEMPT**


In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
with open("/content/info4940-sitcom/episode_5_formatted.txt", "r") as file:
    text = file.read()

# Split the text content based on "The time is"
chunks = text.split("The time is")
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

# Print out the resulting chunks
for chunk in chunks:
    print(chunk)


00:00:00:220000. The scene is The Cheesecake Factory. The recipients are Leonard, Raj, Howard. The speaker is Sheldon. Sheldon says "All right! I'm moving my infantry division. Augmented by bataillon of orcs from Lord of the Rings. We flank the Tennessee volunteers and the North, once again, wins the battle of Gettysburg."
00:00:12:110000. The scene is The Cheesecake Factory. The recipients are Sheldon, Leonard, Raj. The speaker is Howard. Howard says "Not so fast! Remember, the South still has two infantry divisions, plus Superman and Godzilla."
00:00:18:100000. The scene is The Cheesecake Factory. The recipients are Sheldon, Raj, Howard. The speaker is Leonard. Leonard says "No, orcs are magic. Superman is vulnerable to magic. Not to mention you already lost Godzilla to the Illinois cavalry and Hulk."
00:00:26:150000. The scene is The Cheesecake Factory. The recipients are Sheldon, Leonard, Howard. The speaker is Raj. Raj says "Why don't you just have Robert E. Lee charge the line wi

In [ ]:
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#from langchain.embeddings.openai import OpenAIEmbeddings
#os.environ["OPENAI_API_KEY"] = ""
#embeddings = OpenAIEmbeddings()

# Create Document objects from the text chunks
documents = [Document(chunk, metadata={"source": "custom_source"}) for chunk in chunks]

# Now you can proceed with creating the vector store
vector_store = FAISS.from_documents(documents, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(documents)

[<__main__.Document object at 0x7b66a9b89db0>, <__main__.Document object at 0x7b66a9b88250>, <__main__.Document object at 0x7b667f8c2a40>, <__main__.Document object at 0x7b667f8c2d70>, <__main__.Document object at 0x7b667f8c3e50>, <__main__.Document object at 0x7b667f8c2ce0>, <__main__.Document object at 0x7b667f8c33a0>, <__main__.Document object at 0x7b667f8c2bf0>, <__main__.Document object at 0x7b667f8c2b90>, <__main__.Document object at 0x7b667f8c3790>, <__main__.Document object at 0x7b667caadde0>, <__main__.Document object at 0x7b667caadd80>, <__main__.Document object at 0x7b667caae200>, <__main__.Document object at 0x7b667caae290>, <__main__.Document object at 0x7b667caae2f0>, <__main__.Document object at 0x7b667caae3e0>, <__main__.Document object at 0x7b667caad5d0>, <__main__.Document object at 0x7b667caadb40>, <__main__.Document object at 0x7b667caada80>, <__main__.Document object at 0x7b667caad1e0>, <__main__.Document object at 0x7b667caad150>, <__main__.Document object at 0x7b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

Mounted at /content/drive


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

In [ ]:
!pip install -qU langchain-openai
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import getpass
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
#testing if the model even works at all
from langchain import PromptTemplate, LLMChain

set_verbose(True)
template = """<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not.
Evaluate the dialogue below from context below. Explain your reason. Output should be in the format of "isHumor: True/False" or "Explanation:".

Context: {context}
Dialogue: {dialogue}
[/INST] </s>
"""

def generate_response(dialogue, context):
  prompt = PromptTemplate(template=template, input_variables=["dialogue", "context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"dialogue":dialogue, "context":context})
  return response

generate_response("""Of course, I'm listening. Blah, blah, hopeless Penny delusion...""",
"""00:08:01:010000. The scene is The hallway, Sheldon scuttles out of apartment door and crosses to Penny’s. Knocks on it urgently. The recipients are Sheldon. The speaker is Penny. Penny says "Okay, sweetie, I know you think you're explaining yourself, but you're really not."
00:13:56:010000. The scene is The hallway. The recipients are Leonard, Sheldon. The speaker is Penny. Penny says "So, is it serious? Do you like her?""")



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. Explain your reason. Output should be in the format of "isHumor: True/False" or "Explanation:".

Context: 00:08:01:010000. The scene is The hallway, Sheldon scuttles out of apartment door and crosses to Penny’s. Knocks on it urgently. The recipients are Sheldon. The speaker is Penny. Penny says "Okay, sweetie, I know you think you're explaining yourself, but you're really not."
00:13:56:010000. The scene is The hallway. The recipients are Leonard, Sheldon. The speaker is Penny. Penny says "So, is it serious? Do you like her?
Dialogue: Of course, I'm listening. Blah, blah, hopeless Penny delusion...
[/INST] </s>



Llama.generate: prefix-match hit


KeyboardInterrupt: 

In [ ]:
#second option for evaluation

import json
import random

# Load episode data
episode_file = "/content/info4940-sitcom/cleaned-data/S1/The Big Bang_S0105.json"
with open(episode_file, 'r') as file:
    episode_data = json.load(file)

# Check and sample 10 from each, ensure enough data is present
if len(humorous) >= 10 and len(not_humorous) >= 10:
    random.shuffle(humorous)
    random.shuffle(not_humorous)
    balanced_dataset = [(True, dialogue) for dialogue in humorous[:10]] + [(False, dialogue) for dialogue in not_humorous[:10]]
    humorous_dataset = [(True, dialogue) for dialogue in humorous[:20]]
    not_humorous_dataset = [(False, dialogue) for dialogue in not_humorous[:10]]
    random.shuffle(balanced_dataset)  # Shuffle to mix humorous and non-humorous dialogues
    random.shuffle(humorous_dataset)
    random.shuffle(not_humorous_dataset)
else:
    print("Not enough data to sample 10 from each category.")
    # Handle error or exit

# Print dataset size
print(f"Dataset size: {len(humorous_dataset)}")

set_verbose(True)
template = """<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not.
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: {context}
Dialogue: {dialogue}
[/INST] </s>
"""

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

def generate_response(dialogue):
    # Use retriever to get context
    context_documents = retriever.get_relevant_documents(dialogue)
    # Filter out the exact dialogue from the context
    filtered_documents = context_documents[1:3]
    context = "\n".join([doc.page_content for doc in filtered_documents])


    prompt = PromptTemplate(template=template, input_variables=["dialogue", "context"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)  # Assuming 'llm' is already defined
    response = llm_chain.run({"dialogue": dialogue, "context": context})
    return response

import re
def extract_classification_and_compare(response_string, actual_is_humorous):
    # Updated regex to handle 'true', 'false' in both lowercase and uppercase, and possibly quoted
    match = re.search(r'(Classification|isHumor):\s*"?(true|false|True|False)"?', response_string, re.IGNORECASE)
    if match:
        predicted_is_humorous = match.group(2).lower() == 'true'
    else:
        predicted_is_humorous = False
    is_correct = predicted_is_humorous == actual_is_humorous
    return is_correct

correct_predictions = 0
total_predictions = 0

# Loop over balanced dataset and evaluate each dialogue
for is_humorous, dialogue in humorous_dataset:
    predicted_humor = generate_response(dialogue)
    print(type(predicted_humor))

    actual_label = "Humorous" if is_humorous else "Not Humorous"

    print(f"Predicted: {predicted_humor}")
    print(f"Actual: {actual_label}\n")

    # Call the classification extraction and comparison function
    is_correct_or_not = extract_classification_and_compare(predicted_humor, is_humorous)
    print("Prediction Correct:", is_correct_or_not)  # Print the correctness of each prediction

    if is_correct_or_not:
        correct_predictions += 1
    total_predictions += 1


accuracy = correct_predictions / total_predictions

# Output the metrics
print(f"\nCorrect Predictions: {correct_predictions}")
print(f"Total Predictions: {total_predictions}")
print(f"Accuracy: {accuracy:.2f}")

Dataset size: 20


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: 00:13:14:050000. The scene is The hallway, Sheldon scuttles out of apartment door and crosses to Penny’s. Knocks on it urgently.. The recipients are Leonard. The speaker is Sheldon. Sheldon says "Inconsiderate. That is the adjective, inconsiderate. "
00:13:40:220000. The scene is The hallway.. The recipients are Sheldon, Penny. The speaker is Leonard. Leonard says "Pretty, very... there's really no objective scale for delineating variations of good. Why do you ask?"
Dialogue:

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3030.02 ms
llama_print_timings:      sample time =      33.12 ms /    57 runs   (    0.58 ms per token,  1721.27 tokens per second)
llama_print_timings: prompt eval time =   42815.85 ms /   189 tokens (  226.54 ms per token,     4.41 tokens per second)
llama_print_timings:        eval time =    5611.07 ms /    56 runs   (  100.20 ms per token,     9.98 tokens per second)
llama_print_timings:       total time =   19848.09 ms /   245 tokens
Llama.generate: prefix-match hit



> Finished chain.
<class 'str'>
Predicted: ```json
{
  "Classification": "False",
  "Explanation": "The dialogue does not seem to be humorous. It is a brief and straightforward exchange between two characters discussing the meaning of an adjective."
}
```
Actual: Humorous

Prediction Correct: False


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: 00:17:31:120000. The scene is Leonard and Lesley’s lab.. The recipients are Howard, Raj. The speaker is Leonard. Leonard says "How did it get on the Internet?"
00:16:21:060000. The scene is Leonar


llama_print_timings:        load time =    3030.02 ms
llama_print_timings:      sample time =      54.77 ms /    93 runs   (    0.59 ms per token,  1698.04 tokens per second)
llama_print_timings: prompt eval time =   10620.20 ms /   130 tokens (   81.69 ms per token,    12.24 tokens per second)
llama_print_timings:        eval time =    9695.73 ms /    92 runs   (  105.39 ms per token,     9.49 tokens per second)
llama_print_timings:       total time =   20715.61 ms /   222 tokens
Llama.generate: prefix-match hit



> Finished chain.
<class 'str'>
Predicted: {
"Classification": "isHumor: True",
"Explanation": "The dialogue is humorous because Leonard is making a reference to a previous incident in which they discovered that a banana was being used to conduct research on the internet. The punchline of 'I'm not sure, but I think I'm about to discover how the banana felt' adds an unexpected twist and humor to the situation."
}
Actual: Humorous

Prediction Correct: True


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: 00:15:01:230000. The scene is The hal

...


> Finished chain.
<class 'str'>
Predicted: {
"Classification": false,
"Explanation": "This dialogue does not have any humor in it as it lacks any punchline or unexpected twist."
}
Actual: Humorous

Prediction Correct: False


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: 00:16:29:060000. The scene is Leonard and Lesley’s lab.. The recipients are Leonard. The speaker is Lesley. Lesley says "Listen, neither of us are neuroscientists, but we both understand the biochemistry of sex. I mean, dopamine in our brains is released across synapses, 


llama_print_timings:        load time =    3030.02 ms
llama_print_timings:      sample time =      35.37 ms /    60 runs   (    0.59 ms per token,  1696.50 tokens per second)
llama_print_timings: prompt eval time =   16963.46 ms /   215 tokens (   78.90 ms per token,    12.67 tokens per second)
llama_print_timings:        eval time =    6620.38 ms /    59 runs   (  112.21 ms per token,     8.91 tokens per second)
llama_print_timings:       total time =   23889.37 ms /   274 tokens
Llama.generate: prefix-match hit



> Finished chain.
<class 'str'>
Predicted: ```json
{
    "Classification": false,
    "Explanation": "The dialogue is not humorous as it deals with serious and potentially inappropriate topics. It is more of a suggestive or flirty statement than a joke."
}
```
Actual: Humorous

Prediction Correct: False


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] You are a humor/joke classification assistant for a sitcom show. You will be provided a dialogue from a sitcom and you must analyze whether it is humorous or not. 
Evaluate the dialogue below from context below. For the classification, you must output `isHumor: True` if you think the dialogue is funny or `isHumor: False`. Return the 'Classification' (isHumor = True or isHumor = False) and 'Explanation' in JSON format.


Context: 00:15:01:230000. The scene is The hallway.. The recipients are Leonard. The speaker is Sheldon. Sheldon says "She's not that intelligent."
00:19:19:100000. The scene is The Cheesecake Factory


llama_print_timings:        load time =    3030.02 ms
llama_print_timings:      sample time =      21.66 ms /    38 runs   (    0.57 ms per token,  1754.30 tokens per second)
llama_print_timings: prompt eval time =    6533.61 ms /   114 tokens (   57.31 ms per token,    17.45 tokens per second)
llama_print_timings:        eval time =    4213.97 ms /    37 runs   (  113.89 ms per token,     8.78 tokens per second)
llama_print_timings:       total time =   10931.54 ms /   151 tokens



> Finished chain.
<class 'str'>
Predicted: {
"Classification": False,
"Explanation": "The dialogue provided does not contain any humor or jokes. It is simply a conversation between two characters."
}
Actual: Humorous

Prediction Correct: False

Correct Predictions: 4
Total Predictions: 20
Accuracy: 0.20
